# Control de la interfaces HDMI

Este cuaderno de Jupyter presenta un tutorial con la información necesaria para interactuar con las bibliotecas de vídeo del entorno de productividad PYNQ-Z2. Para esta práctica se requiere el siguiente hardware:

* 2 cables HDMI
* 1 monitor con entrada HDMI

En primer lugar, se hace el llamado a las bibliotecas del entorno de productividad PYNQ relacionadas con el control de video. A continuación, se declaran las variables para el control de los puertos HDMI de entrada y de salida.

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("base.bit")
hdmi_in  = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

## Configuración del hardware

El objetivo de la práctica consiste en que la placa capture una imagen por el puerto HDMI de entrada y mostrar versiones de dicha imagen en el monitor usando el puerto HDMI de salida. La configuración del hardware se realiza de la siguiente forma:

* Conecte un cable desde una salida HDMI del ordenador hasta el puerto de entrada HDMI de la placa (HDMI IN). 
* Conecte el otro cable desde el puerto de salida HDMI de la placa (HDMI OUT) al monitor. 

En principio, se usa el formato de vídeo por defecto que consta de 24 bits por píxel BGR (*blue, green, red *) para poder usar la biblioteca OpenCV. Al ejecutarse la celda de código mostrada a continuación, se debe encender el monitor y mostrar la pantalla en negro.

In [2]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

La forma más sencilla de desplegar la imagen que recibe la placa por el puerto HDMI IN en el monitor consiste en usar el método *tie* como se muestra a continuación.

In [3]:
hdmi_in.tie(hdmi_out)

Esta imagen se mostrará hasta que se envíe otra imagen o hasta que los puertos HDMI se cierren. Este método provee una forma rápida de pasar datos de vídeo a través de la placa a la pantalla de un monitor. Sin embargo, el método *tie* no permite modificar los frames. Con el objeto de mirar con más detalle la gestión de la librería de vídeo, usaremos los métodos `readframe` y `writeframe` en un lazo de repetición.

In [4]:
import time

numcuadros = 600
inicio = time.time()

for _ in range(numcuadros):
    # asigna la variable f la imagen capturada en HDMI IN
    f = hdmi_in.readframe()
    # envía la imagen al monitor mediante HDMI OUT
    hdmi_out.writeframe(f)
fin = time.time()
print("Cuadros por segundo:  " + str(numcuadros // (fin - inicio)))

Cuadros por segundo:  34.0


Ahora se plicará un filtro de suavizado a los cuadros usando una biblioteca de OpenCV codificada en Python. En particular, el cuadro capturado se convierte en una imagen de escala de grises, y luego se aplica un filtro de suavizado a la versión de escala de grises de la imagen. La versión suavizada se muestra en pantalla. 

In [5]:
import cv2
import numpy as np

numcuadros = 10
grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)
inicio = time.time()

for _ in range(numcuadros):
    # Convierte a grayscale
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)
    inframe.freebuffer()
    # Filtrado de imagen
    cv2.blur(grayscale, (5,5), dst=result)

    outframe = hdmi_out.newframe()
    # Convierte a BGR para poder mostrar en pantalla
    cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe)
    hdmi_out.writeframe(outframe)    
fin = time.time()
print("Cuadros por segundo:  " + str(int(numcuadros / (fin - inicio))))

Cuadros por segundo:  5


## Cierre de los puertos HDMI

Los puertos HDMI deben cerrarse una vez finalizada la tarea que deben realizar. Para cerrar los puertos, use el código siguiente:

In [6]:
hdmi_out.close()
hdmi_in.close()

## Cuadros en la memoria cache de la CPU

Por defecto, la propiedad de `cacheable_frames` de los puertos HDMI tienen asignado el valor de `True` que consiste en que la memoria cache del CPU estará disponible para acelerar las operaciones de software a expensas de tener que borrar cuadros de antes de entregarlos al sistema de vídeo. Al cambiar esta propiedad al valor `False`, se otorga prioridad en el uso de la memoria cache del CPU a la transmisión de cuadros de vídeo, penalizando el rendimiendo de las bibliotecas de software con un desempeño más lento o en algunos casos impidiendo su operación.

In [7]:
base.download()

hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)
hdmi_out.cacheable_frames = False
hdmi_in.cacheable_frames = False
hdmi_out.start()
hdmi_in.start()

Ejecute la siguiente celda de código que es el usado en celdas previas para mostrar imágenes en el monitor y compare su rendimiento.

In [8]:
numcuadros = 600
inicio = time.time()

for _ in range(numcuadros):
    # asigna la variable f la imagen capturada en HDMI IN
    f = hdmi_in.readframe()
    # envía la imagen al monitor mediante HDMI OUT
    hdmi_out.writeframe(f)
fin = time.time()
print("Cuadros por segundo:  " + str(numcuadros // (fin - inicio)))

Cuadros por segundo:  59.0


Ahora veamos cómo afecta esta propiedad al rendimiento de la biblioteca OpenCV en el suavizado de los cuadros.

In [9]:
numcuadros = 10
grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)
inicio = time.time()

for _ in range(numcuadros):
    # Convierte a grayscale
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)
    inframe.freebuffer()
    # Filtrado de imagen
    cv2.blur(grayscale, (5,5), dst=result)

    outframe = hdmi_out.newframe()
    # Convierte a BGR para poder mostrar en pantalla
    cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe)
    hdmi_out.writeframe(outframe)    
fin = time.time()
print("Cuadros por segundo:  " + str(int(numcuadros / (fin - inicio))))

Cuadros por segundo:  1


In [10]:
hdmi_out.close()
hdmi_in.close()

## Imágenes en escalas de grises

El uso del formato en escala de grises en el entorno PYNQ puede mejorar el rendimiento de las aplicaciones diseñadas debido a que la conversión de color se realiza en hardware. Además, se puede transmitir/recibir el valor de un píxel en formato de escala de grises al/desde el CPU. En la celda de código mostrada a continuación, en principio, se reconfiguran los puertos HDMI para que usen el formato de escala de grises. Luego, se asigna el valor por defecto `True` a la propiedad `cacheable_frames` de los puertos HDMI de entrada y salida. En otras palabras, se le otorga prioridad a las operaciones de sofware a expensas del rendimiento del sistema de vídeo. Finalmente, para verificar su correcto funcionamiento, se usa el método `tie` para mostrar en el monitor la imagen que captura la placa en formato de escala de grises. 

In [11]:
base.download()

hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)
hdmi_in.cacheable_frames = True
hdmi_out.cacheable_frames = True
hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

Ejecute la celda de código siguiente y compare su rendimiento con el código ejecutado usando el formato de color.

In [12]:
numcuadros = 30
inicio = time.time()

for _ in range(numcuadros):
    # Captura directamente es escala de grises
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    cv2.blur(inframe, (5,5), dst=outframe)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)

fin = time.time()
print("Cuadros por segundo:  " + str(int(numcuadros / (fin - inicio))))

Cuadros por segundo:  12


In [13]:
hdmi_out.close()
hdmi_in.close()

## Next Steps

This notebook has only provided an overview of base overlay pipeline. One of the reasons for the changes was to make it easier to add hardware accelerated functions by supporting a wider range of pixel formats without software conversion and separating out the HDMI front end from the video DMA. Explore the code in pynq/lib/video.py for more details.